In [1]:
import tensorflow as tf
import numpy as np
import sys

from sklearn.metrics import mean_squared_error
from keras.callbacks import LearningRateScheduler

sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')

#Keras modules
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_Oscillator import OscillatorDataHandler

#Custom modules
from ann_framework import aux_functions

import aux_functions_stochastic
import analytic_functions
import loss_functions

#global variables

deltas = [0.1, 0.1]

k = 1
c = 0.1
D = 1
num_fevals = 5

Using TensorFlow backend.


### Define tensorflow model

In [2]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y


def create_placeholders_additional(input_shape, output_shape):
    X = tf.placeholder(tf.float32, shape=(None, input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=(None), name="y")
    c = tf.placeholder(tf.float32, shape=(), name="c")
    k = tf.placeholder(tf.float32, shape=(), name="k")
    D = tf.placeholder(tf.float32, shape=(), name="D")
    batch_size = tf.placeholder(tf.int32, shape=(), name="batch_size")
    deltas = tf.placeholder(tf.float32, shape=(input_shape), name="deltas")

    return X, y, c, k, D, batch_size, deltas

def tf_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), 
                         name="fc1")
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    
    return y

### Create Model TF

In [3]:
def tf_compiled_model(num_features, output_shape):
    tf.reset_default_graph()

    X, y = create_placeholders(num_features, output_shape)

    y_pred = tf_model(X)
    loss_function = loss_functions.tf_squared_residual_function(k, c, D, deltas, num_fevals)
    cost = loss_function(X, y_pred)
    reg_cost = tf.losses.get_regularization_loss()
    total_cost = cost + reg_cost

    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.5).minimize(total_cost)

    return {'X_placeholder': X, 'y_placeholder': y, 'y_pred': y_pred, 'cost': cost, 'total_cost': total_cost, 'optimizer': optimizer}


def tf_compiled_model_additional(num_features, output_shape):
    tf.reset_default_graph()
    
    X, y, c, k, D, batch_size, deltas = create_placeholders_additional(num_features, output_shape)

    y_pred = tf_model(X)
    cost = loss_functions.squared_residual_function(X, y_pred, deltas, k, c, D, batch_size)
    reg_cost = tf.losses.get_regularization_loss()
    total_cost = cost + reg_cost

    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.5).minimize(total_cost)

    return {'X_placeholder': X, 'y_placeholder': y, 'c_placeholder': c, 'k_placeholder': k, 'D_placeholder': D,
            'batch_size_placeholder': batch_size,
            'deltas_placeholder': deltas, 'y_pred': y_pred, 'cost': cost, 'total_cost': total_cost,
            'optimizer': optimizer}

### Train Functions

In [4]:
def train_model_additional(tf_session, model, X_train, y_train, batch_size, epochs, get_minibatches_function_handle, deltas, k, c,
                D, verbose=1):

    # Retrieve model variables
    X = model['X_placeholder']
    y = model['y_placeholder']
    c_tf = model['c_placeholder']
    k_tf = model['k_placeholder']
    D_tf = model['D_placeholder']
    batch_size_tf = model['batch_size_placeholder']
    deltas_tf = model['deltas_placeholder']

    optimizer = model['optimizer']
    total_cost = model['total_cost']
    cost = model['cost']

    avg_cost_reg = 0.0
    avg_cost = 0.0

    with tf_session.as_default():

        # To reset all variables
        tf_session.run(tf.global_variables_initializer())

        for epoch in range(epochs):

            cost_tot = 0.0
            cost_reg_tot = 0.0
            
            X_batches, y_batches, total_batch = get_minibatches_function_handle(X_train, y_train, batch_size, deltas=deltas)

            # Train with the minibatches
            for i in range(total_batch):
                batch_x, batch_y = X_batches[i], y_batches[i]
                batch_size_real = int(batch_x.shape[0] / num_fevals)

                _, c_reg, f_cost = tf_session.run([optimizer, total_cost, cost],
                                     feed_dict={X: batch_x, y: batch_y, k_tf: k, c_tf: c, D_tf: D,
                                                batch_size_tf: batch_size_real, deltas_tf: deltas})
                cost_tot += f_cost
                cost_reg_tot += c_reg

                avg_cost = cost_tot / total_batch
                avg_cost_reg = cost_reg_tot / total_batch

            if verbose == 1:
                print("Epoch:", '%04d' % (epoch + 1), "cost_reg=", "{:.9f}".format(avg_cost_reg), "cost=", "{:.9f}".format(avg_cost))

    print("Epoch:Final", "cost_reg=", "{:.9f}".format(avg_cost_reg), "cost=", "{:.9f}".format(avg_cost))


def train_model(tf_session, model, X_train, y_train, batch_size, epochs, get_minibatches_function_handle, deltas, verbose=1):

    # Retrieve model variables
    X = model['X_placeholder']
    y = model['y_placeholder']

    optimizer = model['optimizer']
    total_cost = model['total_cost']
    cost = model['cost']

    avg_cost_reg = 0.0
    avg_cost = 0.0

    with tf_session.as_default():

        # To reset all variables
        tf_session.run(tf.global_variables_initializer())

        for epoch in range(epochs):

            cost_tot = 0.0
            cost_reg_tot = 0.0

            X_batches, y_batches, total_batch = get_minibatches_function_handle(X_train, y_train, batch_size, deltas=deltas)

            # Train with the minibatches
            for i in range(total_batch):
                batch_x, batch_y = X_batches[i], y_batches[i]

                _, c_reg, f_cost = tf_session.run([optimizer, total_cost, cost], feed_dict={X: batch_x, y: batch_y})
                cost_tot += f_cost
                cost_reg_tot += c_reg

                avg_cost = cost_tot / total_batch
                avg_cost_reg = cost_reg_tot / total_batch

            if verbose == 1:
                print("Epoch:", '%04d' % (epoch + 1), "cost_reg=", "{:.9f}".format(avg_cost_reg), "cost=", "{:.9f}".format(avg_cost))

    print("Epoch:Final", "cost_reg=", "{:.9f}".format(avg_cost_reg), "cost=", "{:.9f}".format(avg_cost))

### Prepare data (create data handler and load)

In [5]:
dhandler_stochastic = OscillatorDataHandler()
dhandler_stochastic.load_data(verbose=1, cross_validation_ratio=0.2, x=[0, 0], boundaries=[10, 10], n=[50, 50])
dhandler_stochastic.print_data()

minibatches_function_handle = aux_functions_stochastic.get_minibatches

X_train = dhandler_stochastic.X_train
y_train = dhandler_stochastic.y_train
X_crossVal = dhandler_stochastic.X_crossVal
y_crossVal = dhandler_stochastic.y_crossVal
X_test = dhandler_stochastic.X_test
y_test = dhandler_stochastic.y_test

Loading data. Cros-Validation ratio 0.2
Printing shapes

Training data (X, y)
(7200, 2)
(7200, 1)
Cross-Validation data (X, y)
(1800, 2)
(1800, 1)
Testing data (X, y)
(1000, 2)
(1000, 1)
Printing first 5 elements

Training data (X, y)
[[ -1.51515152   8.98989899]
 [  4.74747475   2.92929293]
 [  0.90909091   0.70707071]
 [ -0.50505051  -4.94949495]
 [  6.56565657 -10.        ]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Cross-Validation data (X, y)
[[-2.12121212 -8.38383838]
 [ 6.36363636 -9.7979798 ]
 [ 0.50505051 -7.17171717]
 [ 9.7979798   2.72727273]
 [10.         -1.71717172]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Testing data (X, y)
[[-8.78787879 -5.35353535]
 [-8.38383838 -8.38383838]
 [ 7.37373737  8.78787879]
 [ 2.92929293  5.75757576]
 [-0.70707071 -1.31313131]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Train with function additional

In [6]:
model_additional = tf_compiled_model_additional(2, 1)

X = model_additional['X_placeholder']
y = model_additional['y_placeholder']
c_tf = model_additional['c_placeholder']
k_tf = model_additional['k_placeholder']
D_tf = model_additional['D_placeholder']
batch_size_tf = model_additional['batch_size_placeholder']
deltas_tf = model_additional['deltas_placeholder']

sess = tf.Session()
#writer = tf.summary.FileWriter('./graphs', sess.graph)

train_model_additional(sess, model_additional, X_train, y_train, 512, 100, 
                       minibatches_function_handle, deltas, k, c, D, verbose=1)
phi_nn = model_additional['y_pred']
phi_pred = sess.run(phi_nn, feed_dict={X: X_crossVal})

#Evaluate real model
sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)
p_real, phi_real = analytic_functions.real_p(X_crossVal[:,0], X_crossVal[:,1], sigma_x, sigma_y)

print("Predicted phi")
print(phi_pred)

print("Real phi")
print(phi_real)

d = 2 * np.pi * sigma_x * sigma_y
c_not = 1/d
p_pred = c_not * np.exp(-phi_pred)

p_pred.flatten()
p_real.flatten()

print("Predicted p")
print(p_pred)

print("Real p")
print(p_real)

i = range(len(phi_real))

for i, phi_r, phi_predicted in zip(i, phi_real, phi_pred):
    print('xy {}, Real Phi {}, Predicted Phi {}'.format(X_crossVal[i], phi_r, phi_predicted))

e_phi = mean_squared_error(phi_pred, phi_real)
print("Cross validation MSE (For phi)")
print(e_phi)

e_p = mean_squared_error(p_pred, p_real)
print("Cross validation MSE (For p)")
print(e_p)

Epoch: 0001 cost_reg= 480.348300680 cost= 468.770448049
Epoch: 0002 cost_reg= 202.942771276 cost= 191.470499452
Epoch: 0003 cost_reg= 106.024066925 cost= 94.625225131
Epoch: 0004 cost_reg= 66.531316566 cost= 55.193386761
Epoch: 0005 cost_reg= 49.754636383 cost= 38.464515241
Epoch: 0006 cost_reg= 41.013288307 cost= 29.773623705
Epoch: 0007 cost_reg= 34.839072609 cost= 23.642713483
Epoch: 0008 cost_reg= 30.578040695 cost= 19.424820272
Epoch: 0009 cost_reg= 27.428244909 cost= 16.315952404
Epoch: 0010 cost_reg= 25.418596331 cost= 14.344959537
Epoch: 0011 cost_reg= 24.019105657 cost= 12.986459057
Epoch: 0012 cost_reg= 23.168436559 cost= 12.170939648
Epoch: 0013 cost_reg= 22.128615061 cost= 11.170932730
Epoch: 0014 cost_reg= 23.083789062 cost= 12.158154090
Epoch: 0015 cost_reg= 22.669623947 cost= 11.779777559
Epoch: 0016 cost_reg= 21.372068914 cost= 10.516628385
Epoch: 0017 cost_reg= 20.574759102 cost= 9.747726019
Epoch: 0018 cost_reg= 20.459273020 cost= 9.664748303
Epoch: 0019 cost_reg= 19.

xy [-5.55555556 -2.72727273], Real Phi [1.9151107], Predicted Phi [0.03275552]
xy [ 7.77777778 -7.57575758], Real Phi [5.8942965], Predicted Phi [0.04199919]
xy [0.1010101  3.33333333], Real Phi [0.55606571], Predicted Phi [0.03715621]
xy [8.38383838 1.51515152], Real Phi [3.62922151], Predicted Phi [0.04215663]
xy [-6.56565657 -6.16161616], Real Phi [4.05366799], Predicted Phi [0.04051446]
xy [-4.94949495  5.75757576], Real Phi [2.88235894], Predicted Phi [0.030224]
xy [-2.92929293 -5.95959596], Real Phi [2.20487705], Predicted Phi [0.04836737]
xy [ 7.97979798 -8.38383838], Real Phi [6.69829609], Predicted Phi [0.04179503]
xy [ 2.32323232 -1.31313131], Real Phi [0.35608611], Predicted Phi [0.04022484]
xy [8.38383838 8.18181818], Real Phi [6.86154474], Predicted Phi [0.01022546]
xy [ 8.18181818 -1.31313131], Real Phi [3.43332313], Predicted Phi [0.04243969]
xy [ 3.53535354 -7.17171717], Real Phi [3.19661259], Predicted Phi [0.03969865]
xy [2.52525253 7.77777778], Real Phi [3.34353637],

xy [ 1.11111111 -4.94949495], Real Phi [1.28660341], Predicted Phi [0.04387682]
xy [-2.72727273 -6.56565657], Real Phi [2.52729313], Predicted Phi [0.04778879]
xy [-5.55555556  2.52525253], Real Phi [1.86205489], Predicted Phi [0.02253285]
xy [ 3.73737374 10.        ], Real Phi [5.69839812], Predicted Phi [0.01759468]
xy [-4.94949495 -7.57575758], Real Phi [4.09448016], Predicted Phi [0.04758686]
xy [ 4.14141414 -9.7979798 ], Real Phi [5.65758596], Predicted Phi [0.04975038]
xy [-5.15151515  6.16161616], Real Phi [3.2251811], Predicted Phi [0.02947757]
xy [ 6.36363636 -7.77777778], Real Phi [5.04948475], Predicted Phi [0.04057481]
xy [ 2.52525253 -4.54545455], Real Phi [1.35190287], Predicted Phi [0.03943769]
xy [1.71717172 1.91919192], Real Phi [0.33159882], Predicted Phi [0.03300004]
xy [-7.77777778 -0.90909091], Real Phi [3.06601367], Predicted Phi [0.03595551]
xy [-1.71717172  1.91919192], Real Phi [0.33159882], Predicted Phi [0.03609036]
xy [ 5.15151515 -8.58585859], Real Phi [5.0

xy [-9.19191919 -1.11111111], Real Phi [4.28629732], Predicted Phi [0.03615163]
xy [ 3.73737374 -6.96969697], Real Phi [3.12723192], Predicted Phi [0.03999216]
xy [-1.71717172 -0.70707071], Real Phi [0.17243138], Predicted Phi [0.03455089]
xy [-7.37373737  8.98989899], Real Phi [6.75951434], Predicted Phi [0.02500593]
xy [-2.12121212 -1.11111111], Real Phi [0.28670544], Predicted Phi [0.03475003]
xy [9.39393939 4.14141414], Real Phi [5.26987042], Predicted Phi [0.02446895]
xy [ 4.14141414 -3.13131313], Real Phi [1.34782165], Predicted Phi [0.04099794]
xy [-1.11111111 -6.96969697], Real Phi [2.49056219], Predicted Phi [0.04579185]
xy [ 3.13131313 -6.16161616], Real Phi [2.38853178], Predicted Phi [0.03958083]
xy [8.58585859 1.31313131], Real Phi [3.77206408], Predicted Phi [0.04187642]
xy [8.78787879 2.72727273], Real Phi [4.23324151], Predicted Phi [0.03677892]
xy [-9.19191919 -4.54545455], Real Phi [5.25762677], Predicted Phi [0.03094529]
xy [-9.39393939 -5.75757576], Real Phi [6.0697

xy [9.19191919 8.58585859], Real Phi [7.9104173], Predicted Phi [0.0084744]
xy [8.78787879 4.34343434], Real Phi [4.80461177], Predicted Phi [0.02083438]
xy [3.73737374 3.13131313], Real Phi [1.18865422], Predicted Phi [0.02804746]
xy [-1.31313131 -3.13131313], Real Phi [0.57647179], Predicted Phi [0.04198706]
xy [ 4.74747475 -7.97979798], Real Phi [4.31078461], Predicted Phi [0.04078915]
xy [-4.14141414  9.5959596 ], Real Phi [5.46168758], Predicted Phi [0.02304985]
xy [9.19191919 7.37373737], Real Phi [6.94316906], Predicted Phi [0.01191973]
xy [ 0.90909091 -5.55555556], Real Phi [1.58453219], Predicted Phi [0.04523506]
xy [-5.15151515  4.34343434], Real Phi [2.27017651], Predicted Phi [0.03259097]
xy [ 5.75757576 -2.32323232], Real Phi [1.92735435], Predicted Phi [0.04306737]
xy [ 2.52525253 -5.75757576], Real Phi [1.97632895], Predicted Phi [0.04359444]
xy [-2.12121212  8.38383838], Real Phi [3.73941435], Predicted Phi [0.03114975]
xy [2.92929293 7.37373737], Real Phi [3.147638], P

### Train with function minimal

In [7]:
model = tf_compiled_model(2, 1)

X = model['X_placeholder']
y = model['y_placeholder']

sess = tf.Session()
#writer = tf.summary.FileWriter('./graphs', sess.graph)

train_model(sess, model, X_train, y_train, 512, 10, minibatches_function_handle, deltas, verbose=1)
phi_nn = model['y_pred']
phi_pred = sess.run(phi_nn, feed_dict={X: X_crossVal})

#Evaluate real model
sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)
p_real, phi_real = analytic_functions.real_p(X_crossVal[:,0], X_crossVal[:,1], sigma_x, sigma_y)

print("Predicted phi")
print(phi_pred)

print("Real phi")
print(phi_real)

d = 2 * np.pi * sigma_x * sigma_y
c_not = 1/d
p_pred = c_not * np.exp(-phi_pred)

p_pred.flatten()
p_real.flatten()

print("Predicted p")
print(p_pred)

print("Real p")
print(p_real)

i = range(len(phi_real))

for i, phi_r, phi_predicted in zip(i, phi_real, phi_pred):
    print('xy {}, Real Phi {}, Predicted Phi {}'.format(X_crossVal[i], phi_r, phi_predicted))

e_phi = mean_squared_error(phi_pred, phi_real)
print("Cross validation MSE (For phi)")
print(e_phi)

e_p = mean_squared_error(p_pred, p_real)
print("Cross validation MSE (For p)")
print(e_p)

Epoch: 0001 cost_reg= 176.777750397 cost= 165.784838867
Epoch: 0002 cost_reg= 108.088999367 cost= 97.399994342
Epoch: 0003 cost_reg= 68.811987813 cost= 58.394070848
Epoch: 0004 cost_reg= 45.568504079 cost= 35.382243252
Epoch: 0005 cost_reg= 32.458363597 cost= 22.490811372
Epoch: 0006 cost_reg= 25.954092026 cost= 16.180771041
Epoch: 0007 cost_reg= 23.092154757 cost= 13.479665196
Epoch: 0008 cost_reg= 20.324902471 cost= 10.862019563
Epoch: 0009 cost_reg= 19.179658190 cost= 9.858623912
Epoch: 0010 cost_reg= 17.390146573 cost= 8.192548227
Epoch:Final cost_reg= 17.390146573 cost= 8.192548227
Predicted phi
[[ 0.01619785]
 [-0.01041145]
 [-0.01640668]
 ...
 [-0.0170958 ]
 [ 0.00247907]
 [-0.00682392]]
Real phi
[[3.73941435]
 [6.82481379]
 [2.58443016]
 ...
 [1.24987246]
 [3.55575962]
 [4.89847975]]
Predicted p
[[0.01565977]
 [0.01608206]
 [0.01617877]
 ...
 [0.01618992]
 [0.01587609]
 [0.01602447]]
Real p
[[3.78279770e-04]
 [1.72918381e-05]
 [1.20064986e-03]
 ...
 [4.56044706e-03]
 [4.5454130

xy [ 4.14141414 -4.54545455], Real Phi [1.89062341], Predicted Phi [-0.02179126]
xy [ 4.14141414 -1.71717172], Real Phi [1.00499949], Predicted Phi [-0.00094042]
xy [-2.12121212  9.19191919], Real Phi [4.44954596], Predicted Phi [-0.04088837]
xy [5.75757576 3.33333333], Real Phi [2.21303949], Predicted Phi [-0.00068708]
xy [ 0.3030303  -4.74747475], Real Phi [1.13151719], Predicted Phi [-0.01306753]
xy [-9.5959596  -5.55555556], Real Phi [6.1473319], Predicted Phi [0.04051121]
xy [-7.17171717 -7.57575758], Real Phi [5.4412815], Predicted Phi [0.03732014]
xy [-2.12121212  7.17171717], Real Phi [2.7966534], Predicted Phi [-0.02800803]
xy [8.58585859 2.32323232], Real Phi [3.9557188], Predicted Phi [-0.02709237]
xy [-10.           2.12121212], Real Phi [5.22497704], Predicted Phi [-0.00954321]
xy [ 6.56565657 -9.5959596 ], Real Phi [6.75951434], Predicted Phi [-0.00835574]
xy [ 8.18181818 -8.78787879], Real Phi [7.20844812], Predicted Phi [-0.06154342]
xy [6.16161616 0.90909091], Real Phi

xy [10.          2.32323232], Real Phi [5.26987042], Predicted Phi [-0.03542051]
xy [-8.98989899  8.58585859], Real Phi [7.72676258], Predicted Phi [0.00905065]
xy [3.93939394 7.17171717], Real Phi [3.34761759], Predicted Phi [-0.02967401]
xy [ 3.13131313 -0.50505051], Real Phi [0.5030099], Predicted Phi [-0.01104449]
xy [6.16161616 2.52525253], Real Phi [2.2171207], Predicted Phi [-0.01086563]
xy [-5.15151515 -0.3030303 ], Real Phi [1.33149679], Predicted Phi [-0.00282075]
xy [-2.12121212 -8.98989899], Real Phi [4.26589124], Predicted Phi [0.01604995]
xy [-0.90909091 -0.50505051], Real Phi [0.05407611], Predicted Phi [-0.00697287]
xy [-10.          -8.38383838], Real Phi [8.5144373], Predicted Phi [0.05032692]
xy [ 8.58585859 -0.70707071], Real Phi [3.71084583], Predicted Phi [-0.02922223]
xy [ -5.15151515 -10.        ], Real Phi [6.32690542], Predicted Phi [0.03683354]
xy [ 7.37373737 -4.34343434], Real Phi [3.66187124], Predicted Phi [0.00387183]
xy [ 8.18181818 -8.98989899], Real P

xy [3.73737374 1.51515152], Real Phi [0.81318233], Predicted Phi [-0.00633731]
xy [-6.96969697 -1.71717172], Real Phi [2.57626773], Predicted Phi [0.01987098]
xy [-5.95959596  1.91919192], Real Phi [1.96000408], Predicted Phi [-0.01807076]
xy [ 4.34343434 -2.72727273], Real Phi [1.31517192], Predicted Phi [0.00886969]
xy [-0.1010101  -8.58585859], Real Phi [3.68635853], Predicted Phi [-0.00611531]
xy [-1.51515152  4.54545455], Real Phi [1.14784206], Predicted Phi [-0.0160394]
xy [-1.31313131 -8.38383838], Real Phi [3.60065299], Predicted Phi [0.00597022]
xy [8.18181818 0.50505051], Real Phi [3.35986124], Predicted Phi [-0.02794591]
xy [2.92929293 6.16161616], Real Phi [2.32731354], Predicted Phi [-0.03196268]
xy [-1.11111111 -5.55555556], Real Phi [1.60493827], Predicted Phi [0.00410326]
xy [7.97979798 8.18181818], Real Phi [6.53096623], Predicted Phi [0.02318483]
xy [-6.36363636  6.56565657], Real Phi [4.1801857], Predicted Phi [0.00167824]
xy [-3.33333333 -4.14141414], Real Phi [1.41

xy [ 0.1010101  -5.75757576], Real Phi [1.65799408], Predicted Phi [-0.00962008]
xy [-1.11111111  7.97979798], Real Phi [3.24558718], Predicted Phi [-0.03580048]
xy [8.18181818 9.19191919], Real Phi [7.57167636], Predicted Phi [0.01198781]
xy [ 0.50505051 -5.95959596], Real Phi [1.788593], Predicted Phi [-0.016445]
xy [-9.19191919 -2.92929293], Real Phi [4.65360677], Predicted Phi [0.03128806]
xy [ 5.15151515 -5.95959596], Real Phi [3.10274462], Predicted Phi [-0.02221855]
xy [ 2.92929293 -2.32323232], Real Phi [0.69890827], Predicted Phi [0.01568422]
xy [1.71717172 0.50505051], Real Phi [0.16018774], Predicted Phi [-0.00418853]
xy [-5.15151515 -3.73737374], Real Phi [2.02530354], Predicted Phi [0.0184634]
xy [ 3.73737374 -0.50505051], Real Phi [0.71115192], Predicted Phi [-0.01306552]
xy [ 0.1010101  -7.97979798], Real Phi [3.18436894], Predicted Phi [-0.00954995]
xy [-10.           5.35353535], Real Phi [6.43301704], Predicted Phi [0.00605309]
xy [ 1.71717172 -4.54545455], Real Phi [

xy [1.71717172 3.53535354], Real Phi [0.77237017], Predicted Phi [-0.01431339]
xy [-4.94949495 10.        ], Real Phi [6.22487501], Predicted Phi [-0.02603777]
xy [0.50505051 3.53535354], Real Phi [0.63769003], Predicted Phi [-0.01875436]
xy [8.98989899 3.13131313], Real Phi [4.53117029], Predicted Phi [-0.02167291]
xy [ 4.94949495 -0.50505051], Real Phi [1.23762881], Predicted Phi [-0.01710764]
xy [7.97979798 4.74747475], Real Phi [4.31078461], Predicted Phi [-0.00012607]
xy [ 3.73737374 -6.36363636], Real Phi [2.72319151], Predicted Phi [-0.00797794]
xy [ 4.54545455 -2.32323232], Real Phi [1.30292827], Predicted Phi [0.00153753]
xy [-5.15151515  2.52525253], Real Phi [1.64575043], Predicted Phi [-0.00494757]
xy [0.90909091 7.17171717], Real Phi [2.61299867], Predicted Phi [-0.02954653]
xy [ 5.95959596 -3.33333333], Real Phi [2.33139476], Predicted Phi [0.00207441]
xy [-2.92929293  7.37373737], Real Phi [3.147638], Predicted Phi [-0.02170835]
xy [ 9.39393939 -3.53535354], Real Phi [5.

xy [3.93939394 4.14141414], Real Phi [1.63350679], Predicted Phi [0.01403062]
xy [0.90909091 3.73737374], Real Phi [0.73972044], Predicted Phi [-0.013998]
xy [-9.5959596  -7.57575758], Real Phi [7.47372717], Predicted Phi [0.04643719]
xy [-1.51515152 -8.58585859], Real Phi [3.80063259], Predicted Phi [0.00847782]
xy [ 6.96969697 -8.78787879], Real Phi [6.29017447], Predicted Phi [-0.01601352]
xy [-3.53535354  3.13131313], Real Phi [1.11519233], Predicted Phi [-0.00613846]
xy [-0.3030303   2.92929293], Real Phi [0.43362922], Predicted Phi [-0.0209506]
xy [-9.5959596   5.95959596], Real Phi [6.37996123], Predicted Phi [0.00907604]
xy [-5.95959596  0.3030303 ], Real Phi [1.78043057], Predicted Phi [-0.00508998]
xy [-0.90909091  2.12121212], Real Phi [0.26629936], Predicted Phi [-0.01399248]
xy [10.         -8.58585859], Real Phi [8.68584838], Predicted Phi [-0.00140712]
xy [-7.17171717 -2.92929293], Real Phi [3.00071421], Predicted Phi [0.02369061]
xy [-0.90909091  5.15151515], Real Phi [

### Create Tunable Model Tensorflow and assign data

In [8]:
tModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', 
                                        data_handler=dhandler_stochastic)

tModel.X_test = dhandler_stochastic.X_test
tModel.X_train = dhandler_stochastic.X_train
tModel.X_crossVal = dhandler_stochastic.X_crossVal
tModel.y_crossVal = dhandler_stochastic.y_crossVal
tModel.y_test = dhandler_stochastic.y_test
tModel.y_train = dhandler_stochastic.y_train

tModel.print_data()

Printing shapes

Training data (X, y)
(7200, 2)
(7200, 1)
Cross-Validation data (X, y)
(1800, 2)
(1800, 1)
Testing data (X, y)
(1000, 2)
(1000, 1)
Printing first 5 elements

Training data (X, y)
[[ -1.51515152   8.98989899]
 [  4.74747475   2.92929293]
 [  0.90909091   0.70707071]
 [ -0.50505051  -4.94949495]
 [  6.56565657 -10.        ]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Cross-Validation data (X, y)
[[-2.12121212 -8.38383838]
 [ 6.36363636 -9.7979798 ]
 [ 0.50505051 -7.17171717]
 [ 9.7979798   2.72727273]
 [10.         -1.71717172]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Testing data (X, y)
[[-8.78787879 -5.35353535]
 [-8.38383838 -8.38383838]
 [ 7.37373737  8.78787879]
 [ 2.92929293  5.75757576]
 [-0.70707071 -1.31313131]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


### Train model Tensorflow

In [9]:
tModel.epochs = 100
lrate = LearningRateScheduler(aux_functions.step_decay)
minibatches_function_handle = aux_functions_stochastic.get_minibatches

#tf.get_variable("deltas", initializer=deltas)

sess = tf.Session()

writer = tf.summary.FileWriter('./graphs', sess.graph)

tModel.train_model(tf_session=sess, get_minibatches_function_handle=minibatches_function_handle, 
                   verbose=1, deltas=deltas)

Epoch: 0001 cost_reg= 453.811200078 cost= 442.320640945
Epoch: 0002 cost_reg= 222.401858012 cost= 211.115918795
Epoch: 0003 cost_reg= 105.140379206 cost= 94.009381676
Epoch: 0004 cost_reg= 55.588217036 cost= 44.588636430
Epoch: 0005 cost_reg= 37.120073382 cost= 26.241491278
Epoch: 0006 cost_reg= 29.076338895 cost= 18.323675100
Epoch: 0007 cost_reg= 25.239518929 cost= 14.605778901
Epoch: 0008 cost_reg= 24.616187096 cost= 14.101236598
Epoch: 0009 cost_reg= 23.820245552 cost= 13.414118719
Epoch: 0010 cost_reg= 22.426454226 cost= 12.104552193
Epoch: 0011 cost_reg= 22.627855682 cost= 12.387549873
Epoch: 0012 cost_reg= 22.398695501 cost= 12.240028044
Epoch: 0013 cost_reg= 21.016315842 cost= 10.937256821
Epoch: 0014 cost_reg= 19.538693492 cost= 9.544148870
Epoch: 0015 cost_reg= 18.502802976 cost= 8.581922678
Epoch: 0016 cost_reg= 19.125550969 cost= 9.275767946
Epoch: 0017 cost_reg= 18.821663348 cost= 9.039471749
Epoch: 0018 cost_reg= 16.814462980 cost= 7.103786794
Epoch: 0019 cost_reg= 17.075

In [10]:
#Evaluate real model
X_test = tModel.X_crossVal

sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)
p_real, phi_real = analytic_functions.real_p(X_test[:,0], X_test[:,1], sigma_x, sigma_y)

tModel.y_crossVal = phi_real

#p_real = np.ravel(p_real)
#phi_real = np.ravel(phi_real)

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
#phi_pred = np.ravel(tModel.y_predicted)
phi_pred = tModel.y_predicted

print(mean_squared_error(phi_pred, phi_real))

d = 2 * np.pi * sigma_x * sigma_y
c_not = 1/d
p_pred = c_not * np.exp(-phi_pred)

print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
mse = cScores['mse']
time = tModel.train_time

i = range(len(phi_pred))


for i, phi_real, phi_predicted in zip(i, phi_real, phi_pred):
    print('xy {}, Real Phi {}, Predicted Phi {}'.format(X_test[i], phi_real, phi_predicted))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("MSE: {}".format(mse))
print("Time : {} seconds".format(time))

tensorflow test
[[3.73941435]
 [6.82481379]
 [2.58443016]
 ...
 [1.24987246]
 [3.55575962]
 [4.89847975]]
[[-0.01048124]
 [-0.0126874 ]
 [-0.00907274]
 ...
 [-0.01296749]
 [-0.0526194 ]
 [-0.01264653]]
16.9397770170626
scores
xy [-2.12121212 -8.38383838], Real Phi [3.73941435], Predicted Phi [-0.01048124]
xy [ 6.36363636 -9.7979798 ], Real Phi [6.82481379], Predicted Phi [-0.0126874]
xy [ 0.50505051 -7.17171717], Real Phi [2.58443016], Predicted Phi [-0.00907274]
xy [9.7979798  2.72727273], Real Phi [5.17192123], Predicted Phi [-0.01195076]
xy [10.         -1.71717172], Real Phi [5.14743394], Predicted Phi [-0.0122917]
xy [ 4.14141414 -8.78787879], Real Phi [4.71890623], Predicted Phi [-0.01189286]
xy [8.58585859 8.98989899], Real Phi [7.72676258], Predicted Phi [-0.06052479]
xy [ 4.34343434 -3.53535354], Real Phi [1.56820733], Predicted Phi [-0.01614804]
xy [4.94949495 1.11111111], Real Phi [1.28660341], Predicted Phi [-0.0128155]
xy [ 3.33333333 -8.38383838], Real Phi [4.06999286], P

xy [-1.71717172 -2.12121212], Real Phi [0.37241098], Predicted Phi [-0.01229433]
xy [0.3030303  3.53535354], Real Phi [0.6295276], Predicted Phi [-0.02735541]
xy [ 8.58585859 -1.51515152], Real Phi [3.80063259], Predicted Phi [-0.01248775]
xy [7.57575758 3.93939394], Real Phi [3.64554637], Predicted Phi [-0.04473981]
xy [-2.32323232 -1.31313131], Real Phi [0.35608611], Predicted Phi [-0.01211622]
xy [8.98989899 8.58585859], Real Phi [7.72676258], Predicted Phi [-0.06245628]
xy [0.1010101  3.13131313], Real Phi [0.49076625], Predicted Phi [-0.02572631]
xy [ 2.32323232 -7.77777778], Real Phi [3.29456178], Predicted Phi [-0.00651307]
xy [-3.13131313  3.93939394], Real Phi [1.26619733], Predicted Phi [-0.03946302]
xy [-1.51515152 -0.50505051], Real Phi [0.12753801], Predicted Phi [-0.01121137]
xy [-0.3030303  9.7979798], Real Phi [4.80461177], Predicted Phi [-0.0449164]
xy [6.76767677 2.32323232], Real Phi [2.55994286], Predicted Phi [-0.02483264]
xy [ 6.36363636 -6.96969697], Real Phi [4.

xy [8.58585859 2.52525253], Real Phi [4.0046934], Predicted Phi [-0.01835623]
xy [-2.52525253 -7.57575758], Real Phi [3.18845016], Predicted Phi [-0.00955237]
xy [-7.97979798 -4.34343434], Real Phi [4.12712988], Predicted Phi [-0.00828266]
xy [-3.93939394 -9.19191919], Real Phi [5.00051015], Predicted Phi [-0.00767736]
xy [-1.51515152  3.93939394], Real Phi [0.89072544], Predicted Phi [-0.02958764]
xy [-4.74747475  3.53535354], Real Phi [1.75186205], Predicted Phi [-0.04203886]
xy [-0.3030303 -0.1010101], Real Phi [0.00510152], Predicted Phi [-0.01251436]
xy [3.73737374 8.38383838], Real Phi [4.21283542], Predicted Phi [-0.03957691]
xy [2.32323232 8.58585859], Real Phi [3.9557188], Predicted Phi [-0.03701575]
xy [-9.39393939 -6.76767677], Real Phi [6.70237731], Predicted Phi [-0.00779304]
xy [-8.58585859 -3.13131313], Real Phi [4.17610448], Predicted Phi [-0.00295305]
xy [-4.14141414 -6.36363636], Real Phi [2.88235894], Predicted Phi [-0.00879384]
xy [-9.5959596 -0.1010101], Real Phi [

xy [0.70707071 9.19191919], Real Phi [4.24956637], Predicted Phi [-0.04517454]
xy [-7.77777778 -4.34343434], Real Phi [3.96796245], Predicted Phi [-0.00875708]
xy [5.55555556 6.56565657], Real Phi [3.69860218], Predicted Phi [-0.04583403]
xy [2.92929293 9.5959596 ], Real Phi [5.03315988], Predicted Phi [-0.03869443]
xy [6.36363636 6.36363636], Real Phi [4.04958678], Predicted Phi [-0.05001852]
xy [-5.15151515 -8.58585859], Real Phi [5.0127538], Predicted Phi [-0.00633557]
xy [2.32323232 3.33333333], Real Phi [0.82542598], Predicted Phi [-0.02958456]
xy [1.91919192 3.93939394], Real Phi [0.96010611], Predicted Phi [-0.02822378]
xy [ 5.15151515 -7.77777778], Real Phi [4.35159678], Predicted Phi [-0.01307264]
xy [ 2.72727273 -1.11111111], Real Phi [0.43362922], Predicted Phi [-0.0133357]
xy [ 4.74747475 10.        ], Real Phi [6.12692582], Predicted Phi [-0.04499692]
xy [-5.95959596 -7.57575758], Real Phi [4.64544434], Predicted Phi [-0.00492693]
xy [-5.35353535 -0.1010101 ], Real Phi [1.

xy [-5.15151515 -3.73737374], Real Phi [2.02530354], Predicted Phi [-0.01095854]
xy [ 3.73737374 -0.50505051], Real Phi [0.71115192], Predicted Phi [-0.01313296]
xy [ 0.1010101  -7.97979798], Real Phi [3.18436894], Predicted Phi [-0.00895929]
xy [-10.           5.35353535], Real Phi [6.43301704], Predicted Phi [-0.0677149]
xy [ 1.71717172 -4.54545455], Real Phi [1.18049179], Predicted Phi [-0.01697465]
xy [0.3030303  3.93939394], Real Phi [0.7805326], Predicted Phi [-0.02856878]
xy [ 0.70707071 -3.53535354], Real Phi [0.64993368], Predicted Phi [-0.01146925]
xy [-9.39393939 -9.39393939], Real Phi [8.82460973], Predicted Phi [3.5993755e-05]
xy [ 3.13131313 -4.54545455], Real Phi [1.52331395], Predicted Phi [-0.0133896]
xy [-0.70707071 -7.77777778], Real Phi [3.04968881], Predicted Phi [-0.01006928]
xy [6.56565657 4.34343434], Real Phi [3.0986634], Predicted Phi [-0.04580387]
xy [-8.58585859  5.95959596], Real Phi [5.46168758], Predicted Phi [-0.06163104]
xy [ 7.37373737 -5.15151515], Re

xy [-7.37373737  2.52525253], Real Phi [3.03744516], Predicted Phi [-0.04285157]
xy [ 4.94949495 -6.76767677], Real Phi [3.51494745], Predicted Phi [-0.01476209]
xy [4.14141414 4.74747475], Real Phi [1.98449138], Predicted Phi [-0.03836271]
xy [ 6.76767677 -7.17171717], Real Phi [4.8617488], Predicted Phi [-0.01571535]
xy [-7.57575758 -3.13131313], Real Phi [3.35986124], Predicted Phi [-0.00532606]
xy [3.93939394 5.55555556], Real Phi [2.31915111], Predicted Phi [-0.03825244]
xy [ 1.11111111 -5.15151515], Real Phi [1.38863381], Predicted Phi [-0.00982802]
xy [-4.34343434  5.35353535], Real Phi [2.37628813], Predicted Phi [-0.04823259]
xy [-4.54545455 -2.32323232], Real Phi [1.30292827], Predicted Phi [-0.00984116]
xy [-4.14141414 -0.1010101 ], Real Phi [0.85807571], Predicted Phi [-0.03036079]
xy [-6.56565657 -5.75757576], Real Phi [3.81287624], Predicted Phi [-0.00686056]
xy [ 5.95959596 -9.5959596 ], Real Phi [6.37996123], Predicted Phi [-0.01167783]
xy [-4.94949495 -7.77777778], Rea

In [11]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
c = np.array([[1], [2], [3]])
d = np.array([[4], [5], [6]])

e1 = mean_squared_error(a, b)
e2 = mean_squared_error(c, d)

print(e1)
print(e2)

9.0
9.0
